In [1]:
import numpy as np
import os
import pandas as pd
import pickle
from random import randint
from random import choice
import re
import requests

from PIL import Image
from nltk.tokenize import sent_tokenize
import tracery
from tracery.modifiers import base_english

##  Load the OpenStreetMap Place Data

Code to create the pickle files is in notebook Create Data Files from OSM.  OpenStreetMap data for Venice was downloaded using some weird online tools.  I've blanked it all out.

In [2]:
data_dir = "/home/jupyter/code/venice/data/"

squares = pickle.load(open(data_dir + 'squares_data_list.p', 'rb'))
canals = pickle.load(open(data_dir + 'canals_data_list.p', 'rb'))
walks = pickle.load(open(data_dir + 'walks_data_list.p', 'rb'))
shops_text = pickle.load( open(data_dir + 'shops_data_list.p', 'rb'))

In [3]:
# where is the gpt-2 model running?

api_port = '7777'

## Image Functions : Flickr and Map Snippets

In [8]:
# Needed for the map code:

overlay_images = !ls data/*.png
img = Image.open("data/venice-tourist-map_smaller.jpg")
imgoutput = "images/"

In [7]:
overlay_images

['data/around.png',
 'data/arrow.png',
 'data/bridge.png',
 'data/circle.png',
 'data/square_arrows.png',
 'data/square.png',
 'data/step_arrow.png',
 'data/x.png']

### Map Snippets

In [9]:
## Code to make little crops of the Venice jpg map randomly plus put annotations on

from IPython.display import display # to display images if you want to inline.

# Set up the info used by the map code here
def get_image_crop(img, imgoutput, filenumber):
    width = randint(100, 200)
    height = randint(100, 200)
    location = randint(0, 100)
    imgwidth = img.size[0]
    imgheight = img.size[1]
    left = randint(0, imgwidth-width)
    right = randint(0, imgheight-height)
    box = (left, right, left+width, right+width)
    im1 = img.crop(box)
    im2 = make_overlay(width, height)
    if im2:
        im1.paste(im2, (location, location), im2)
    #display(im1)   # only needed for in notebook testing.
    filename = imgoutput + "image_" + str(filenumber) + ".jpg"
    im1.save(filename)
    return filename
    
def get_map(text, *params):
    global maps
    global img
    global output
    filename = get_image_crop(img, imgoutput, maps)
    maps += 1
    return f"\n<img src='{filename}'>\n"

def make_overlay(width, height):
    # Overlay is scribbled directions like arrows, x's on the map
    image2 = choice(overlay_images)
    rotation = choice([0,90,45,30])
    if image2:
        image = Image.open(image2)
        image_width = image.size[0]
        image_height = image.size[1]
        try:
            newwidth = randint(image_width//5, width)
            newheight = randint(image_height//5,  height)
        except:
            newwidth = width - 50
            newheight = height - 50
        image = image.convert('RGBA')
        image = image.rotate(rotation)
        image = image.resize((newwidth, newheight))
        #display(image)
        return image
    else:
        return None

In [10]:
# Test.
maps = 0
get_map("hi there")

"\n<img src='images/image_0.jpg'>\n"

### Code to look up flickr images based on search

Re-used from old js code of mine.

In [13]:
#flickrkey = os.environ['FLICKR']
# TODO: Remove before posting.
flickrkey = 'e08843a8ba2ec0444013a39cf4082568'

def get_url(tags, key):
    # tags needs to be comma sep string
    baseUrl = f"https://api.flickr.com/services/rest/?method=flickr.photos.search&api_key={key}&per_page=50&format=json&nojsoncallback=1&safe_search=1&content_type=1&tag_mode=all"
    search = f"&tags={tags}"
    return baseUrl + search

def get_flickr_pic(result):
    from random import choice
    if 'photos' in result:
        photo = choice(result['photos']['photo'])
        farmId = photo['farm']
        serverId = photo['server']
        picid = photo['id']
        secret = photo['secret']
        info = get_photo_info(picid, flickrkey)
        author = info['author']
        url = info['url']
        # _z is a size attribute, see https://www.flickr.com/services/api/misc.urls.html
        newurl = f"\n[<img src='http://farm{farmId}.staticflickr.com/{serverId}/{picid}_{secret}_m.jpg' alt='Flickr, by {author}'>]({url})\n"
        return newurl
    else:
        return ' '

def get_photo_info(picid, key):
    url =  f"https://api.flickr.com/services/rest/?method=flickr.photos.getInfo&format=json&nojsoncallback=1&api_key={key}&photo_id={picid}"
    result = requests.get(url)
    res = result.json()
    try:
        author = res['photo']['owner']['username']
    except:
        author = 'No author found.'
    try:
        url = res['photo']['urls']['url'][0]['_content']
    except:
        url = None
    return {'author': author, 'url': url}

In [11]:
## Code to get search strings for flickr pics. Again, not perfect because the final output was done in chunks.

def get_photo(text, *params):
    import os
    global counter
    global length
    
    #flickrkey = os.environ['FLICKR']
    flickrkey = 'e08843a8ba2ec0444013a39cf4082568'
    
    fraction = counter/length
    prefixes = {
        'a': ["Venice square", "Italian church", "Venice fountain", "Venice houses", "Venice cat", 
              "gondola", "Venice bridge"],
        'b': ["Venice dog", "Venice bridge", "Venice glass", "Murano glass", "Italian church", 
              "Venice cafe", "Venice cat", "Venice restaurant", "Venice woman", "Venice shop", "Venice man",
             "Venice festival", "Venice carnival"],
        'c': ["Venice night", "Venice lights", "Venice canal", "Venice reflection", "Venice water", 
              "Venice sunset", "Venice mask", "Venice cat", "Venice church", "Venice market"],
        'd': ["Venice floods", "Venice restaurant", "Venice rain", "Venice night", "Venice bar", 
              "Venice evening", "Venice fog", "Venice storm", "Venice flood", "Venice water", "Venice mask"],
        'e': ["Venice alley", "Venice night", "Venice fishing", "Venice gondola night", "Venice darkness", 
              "Venice clouds", "Venice boat", "Venice restaurant"]
    }
    
    if fraction < .15:
        tags = choice(prefixes['a'])
    elif fraction < .25:
        tags = choice(prefixes['b'])
    elif fraction < .5:
        tags = choice(prefixes['c'])
    elif fraction < .75:
        tags = choice(prefixes['d'])
    elif fraction <= 1:
        tags = choice(prefixes['e'])
    
    title = tags
    tags = ','.join(title.split(' '))
    url = get_url(tags, flickrkey.replace("'","")) # ridiculous need to remove single quotes?
    result = requests.get(url)
    res = result.json()
    if 'photos' in res:
        return get_flickr_pic(res)
    else:
        return f"\n##{title}"

In [14]:
# test
counter = 10
length = 200
get_photo("a quiet day in venice")

"\n[<img src='http://farm66.staticflickr.com/65535/49194480562_3bcd39e634_m.jpg' alt='Flickr, by NiBe60'>](https://www.flickr.com/photos/162684392@N05/49194480562/)\n"

## Code to Run Special Functions Related to GPT-2

###  Call model and process results to clean them.

Issues related to training data and repetition in the results.

In [64]:
# Used for post-processing GPT2 output.


punct = '!"#$%&\'()*+,-/:;<=>?@[\\]^_`{|}~“.'

def remove_bad_dupe_sents(text):
    # first clean fragments internally. we already ended on a period.
    lines = text.splitlines()
    #print("lines coming in", lines)
    #print("number of lines,", len(lines))
    lines = [x for x in lines if len(x.strip()) > 1]  # lines with long enough members
    
    lines = [x for x in lines if (re.search(r'[,\.\:-\?!]', x)) or (x.title() != x)]  # not title case and has punc
    #print("lines after cleaning:", lines)
    text = ' '.join(lines)
    
    no_dupes = []
    sent_tokenize_list = sent_tokenize(text)
    for el in sent_tokenize_list:
        if not el in no_dupes:
            no_dupes.append(el)
    return ' '.join(no_dupes)

def remove_all_caps(text):
    # problem with training data
    # also removed newlines
    words = text.split()
    words = [word for word in words if not (word.isupper() and len(word) > 1)]
    return ' '.join(words)

# Functions used in conjunction with Tracery, as code modifiers.

## This function calls GPT2 and does a bunch to try to rationalize the output.  It often fails with the small model.

def call_model(text, prefix=None, length=None):

    text = text.strip()
    
    #print(f"\n---model called with prefix {prefix} endprefix\n")
    #print(f"\n---model called with text: {text}\n\n")

    req = requests.post('http://0.0.0.0:' + api_port,
                    json={'context': prefix,
                        'seed': text})
    res = req.json()['text'][0]
    
    #print(f"\nRaw model result:{res}\n\n")
    
    if length:
        res = prefix + res[len(text):length] # remove the preceding prompt context but add the prefix
    else:
        res = prefix + res[len(text):]
    
    res = res[:res.rfind('.')+1]  # end at a period.  will actually remove it. Does not work right for St. Mark.
       
    # An artifact of the data training using Gutenberg.
    if 'Illustration' in res:
        res = res.replace('Illustration', '')

    res = remove_bad_dupe_sents(res)  # and other slight cleanup. requires there be \n in it.
    res = remove_all_caps(res)  # remove multichar all text chars.
     
    res = re.sub(r'( +)', ' ', res) # remove excess spaces

    if res:
        return res.rstrip(".")
    else:
        return choice(["--But I forgot where I was going with this.",
                       "--But I've gotten off-topic.", "--But back to the point.", "--Anyway, back to the goal."])


In [31]:
# Test cleaning and dupe removal.
test = """After winding your way through the alleys of Venice, you enter the square the Campo de San Silvestro. It is here that the name of Venice comes to the surface. It was here that the first charter of the city was signed and the first nails were hammered into the foundation of the city.
A
Nave
Piazzetta di San Giorgio
The Piazzetta di San Giorgio, or Piazzetta dei Miracoli, is the
The first altar on the left of the central altar is the ancient and still the best. It is said to have been the first of its kind in the world. In the left transept you’ll find the famous “Ave Maria della Salute”, the original name of the church. Built in the 1220s, it was commissioned by the Doge Francesco Dandolo, who was then a prisoner of the Turks. It was then rebuilt, and in its present form it remained unaltered for almost a hundred years.
"""

remove_bad_dupe_sents(test)

'After winding your way through the alleys of Venice, you enter the square the Campo de San Silvestro. It is here that the name of Venice comes to the surface. It was here that the first charter of the city was signed and the first nails were hammered into the foundation of the city. Piazzetta di San Giorgio The Piazzetta di San Giorgio, or Piazzetta dei Miracoli, is the The first altar on the left of the central altar is the ancient and still the best. It is said to have been the first of its kind in the world. In the left transept you’ll find the famous “Ave Maria della Salute”, the original name of the church. Built in the 1220s, it was commissioned by the Doge Francesco Dandolo, who was then a prisoner of the Turks. It was then rebuilt, and in its present form it remained unaltered for almost a hundred years.'

In [65]:
##  Functions that take text and call GPT2.

def get_stage(text, *params):
    #print("in get_stage with prefix You see:", text)
    final = call_model(text, prefix="You see")
    return final.rstrip(".")

def get_views_desc(text, *params):
    global sentences
    inputtext = ""
    if sentences and len(sentences) > 1:
        inputtext = ' '.join(safe_context_withoutpics(n=1))
    final = call_model(inputtext + f" You see beautiful Venetian views of {text} and", prefix=f"{text} ", length=500)
    return final.rstrip(".")

def get_atmosphere(text, *params):
    global sentences
    inputtext = ""
    prefix = choice(['\nA tip: ', '\nJust an aside: ', '\nReminder here: ', '\nSetting the scene: ', '\nSome thoughts here: ', '\nBy the way: ', 
                    '\nIncidentally: ', '\nSo you know: ', '\nRandom fact: ', '\nTake a note: '])
    
    if sentences and len(sentences) > 1:
        inputtext = ' '.join(safe_context_withoutpics(n=2))
        if inputtext:
            return call_model(inputtext, prefix=prefix).rstrip('.')
    else:
        print(f"no result from model {inputtext, prefix}")
        return choice(["\nIn point of fact: It's a cold day in Venice", 
                      "As a matter of fact, bring your boots, I should warn you",
                       "You might want to bring an umbrella, by the way",
                      "There may be a festival on, so you might need to detour"])

def get_square_desc(text, *params):
    final = call_model(f"After winding your way through the alleys of Venice, you enter the square {text}.", 
                       prefix=f"the square {text}.", length=500)
    return final.rstrip(".")

In [53]:
# Code to get prompt text for GPT2 model based on how far we are thru the total length.  Sets mood a bit.

def get_mood_prefix(text):
    global counter
    global length
    
    fraction = counter/length
    prefixes = {
        'a': ["You've arrived in Venice and you look around you. You see", "You're in Venice now! Everywhere you can see", 
              "The bars and restaurants are full in Venice. You see",
              "People are partying in Venice! You can see joyful", 
              "There are shops selling souvenirs and artwork all around you. You can see",
             "Venice is full of people and history. The canals are full of boats. You see"],
        'b': ["You've been walking a while in Venice and have reached the square. You see",
             "You're at a huge square now, and you see", "In the crowded Venetian square, you see", 
              "There are often Venetian festivals here. You can see", "There is a crowd in Venice today. You see"
              "There is a religious holiday soon, and you can see", 
              "In the nearby shops, women are selling glasswork, and you can see", 
             "There are fishermen returning in Venice, offering their fish at the market. You see"],
        'c': ["After turning left at the church, you reach the canal. It is now dark. Night in Venice is beautiful. You see",
             "The lights of Venice shine across the water. It's beautiful. You can hear and see",
             "The water is lapping quietly against the pier.You feel peaceful as the fog descends. You can smell",
             "The weather is fine in Venice today as the night falls. There is a bit of mist descending. You can see"],
        'd': ["The water is high in Venice, often a problem. It has been raining and the canals are full. The water overflows onto the pavement. You see",
             "You wish you had an umbrella. Venice is wet and dark. You see",
              "Someone in a cloak and mask is following you in the dark canals. You see",
             "The night feels frightening after the carnival in Venice.  You need to hurry past the canals. You see"],
        'e': ["You hurry through the streets of Venice, feeling lost. You've been walking a long time. Is someone following you? You see",
             "The alleys of Venice are narrow. Be careful as you run. You hear someone behind you and see", 
              "Someone with a Venetian mask is chasing you in the dark alleys. You can see"]
    }
    if fraction < .15:
        return choice(prefixes['a'])
    elif fraction < .25:
        return choice(prefixes['b'])
    elif fraction < .5:
        return choice(prefixes['c'])
    elif fraction < .75:
        return choice(prefixes['d'])
    elif fraction <= 1:
        return choice(prefixes['e'])

In [54]:
def safe_context_withoutpics(n=2):
    # Don't choose context sentences with image links in them.
    global sentences
    bigsubset = 15
    safe = [x for x in sentences[-bigsubset:] if 'img src' not in x]
    return safe[-n:]

### Create the Utility Functions for Tracery Use

In [55]:
def get_time(text, *params):
    return str(randint(1, 20))

def get_distance(text, *params):
    return str(randint(1, 500))

def fix_punct(text, *params):
    # At end of generation, add newline.
    return text + "\n"

In [56]:
functions = {
    'get_time': get_time,
    'get_distance': get_distance,
    'get_stage': get_stage,
    'get_prefix': get_mood_prefix,
    'view_desc': get_views_desc,
    'get_atmo': get_atmosphere,
    'get_square_desc': get_square_desc,
    'get_photo': get_photo,
    'get_map': get_map,
    'fix_punct': fix_punct
}

### Tracery Main Grammar

Generates the basic directions, and calls the GPT-2 model for some of them (e.g., square descriptions, atmosphere; and calls picture lookup functions for maps and flickr inserts.)

In [37]:
rules = {
    "origin": ["#story.fix_punct#"],
    "story": ["Now #direction#.", "#direction.capitalize#.", "You will need to #direction#.", "Then, #direction#.", 
              "Now, #direction#.", "#arrive.capitalize#.", "#arrive.capitalize#.", "#boat.capitalize#.", 
              "#boat.capitalize#.", "Don't forget to #direction#.", "#photo#",
              "#atmosphere#.", "#atmosphere#.", "#photo#", "#map#"],
    "direction": ["#dir# #where#", "#dir# #howlong#", "#boat#", "#take#"],
    "dir": ["turn #compass#", "go #compass#", "take a #compass#", "veer #compass#", "#straight#"],
    "take": ["take the #nth# #compass#", "take the #nth# #compass#", "take the #nth# turn #compass#",
            "after the #nth# turn, #dir#", "pass the #nth# alleyway", "go by the #nth# door and #dir#"],
    "nth": ['first', 'second', 'third', 'fourth', 'fifth', 'sixth', 'seventh', 'eighth', 'ninth', 'tenth'],
    "compass": ["left #place#", "right #place#",],
    "straight": ["go straight ahead", "go ahead", "go straight on for #time.get_time# minutes", 
                 "walk without turning around"],
    "howlong": ["after #modifier# #distance.get_distance# meters", "and go for #modifier# #time.get_time# minutes", 
                "for #modifier# #time.get_time# minutes, #how#"],
    "how": ["walking quickly", "while meandering to admire the views of #views of.view_desc#",
            "striding swiftly", "fighting through the crowds here", "where you can smell #smells of#",
            "trying not to think of #smells of#", "enjoying the views of #views of.view_desc#"],
    "where": ["over the bridge", "past #shop#", "beyond #shop#, where a cat watches from the window #catdetail#",
              "through the tour group", "and just past the cat who sits there #catdetail#", 
              "and go past the dogs, #dogdetail#", "where you can smell #smells of#",
             "past #squares#, #square_desc#", "through #squares# without stopping"],
    "modifier": ["exactly", "about", "approximately", "no less than", "at least", "precisely", "around"],
    "boat": ["take a boat at the #canals#. #boatstuff#", "take a gondola at #canals#. #boatstuff#", 
             "hire a boat at #canals#. #boatstuff#", "look for a water taxi when you reach #canals#"],
    "arrive": ["you will come to #shop#. #setting.get_stage#", "you will see #shop#. #setting.get_stage#", 
               "now you will arrive at #canals#. #setting.get_stage#", "you will find #roads#. #setting.get_stage#", 
               "#atmosphere.capitalize#", "you will enter #squares.get_square_desc#",
               "here is a nice big square, #squares.get_square_desc#",
               "you will find yourself in #squares.get_square_desc#",
              "you now find yourself in #squares.get_square_desc#",
               "here is a nice big square, #squares.get_square_desc#"],
    "views of": ["the church,", "the canal,", "the piazza,", "the children playing,", "the little gondolas,", 
                 "the islands,", "the glasswork souvenirs", "souvenir shops selling masks", "art shops",
                "the people in the cafes,", "the colorful houses,", "the old doors,", "the ancient signs,", 
                 "the old stone well,"],
    "smells of": ["the ocean", "fish", "frying foods", "pasta", "rain", "mold", "pastries", "wet clothes"],
    "place": ["on #canals#", "onto #roads#", "after the passageway #roads#"],
    "catdetail": ["(is this a cat you have seen before?)", "(this cat seems familiar)", "(this cat is #catcolor#)",
                  "(this #catcolor# cat reminds you of another cat)", "(is this cat following you?)"],
    "dogdetail": ["without petting them", "--ok these are ones you can pet", "which might bark at you", 
                  "who might follow you for a while", "who are playing with the children", 
                  "which are not dangerous"],
    "catcolor": ["orange", "tortoise-shell", "gray", "black", "striped", "white", "slightly transparent", 
                 "smiling"],
    "boatstuff": ["Tell your pilot to take you to the stop closest to #shop#", "Show them your money, first", 
                  "While you relax in the boat, admire #views of.view_desc#",
                  "Get off the boat after #time.get_time# minutes", "Look for the cheaper water taxis first",
                  "Try to find the gondolier with the #clothing_color# #clothing#",
                 "Get out when you reach the #roads#", "Ask to be let off at #shop#"],
    "square_desc": ["where there is a covered stone well", "where there are empty benches", 
                    "where the benches are full of old women watching everything",
                    "where there is fountain running into a plastic bucket", "where there is laundry hanging overhead",
                   "where children are playing", "where old men are arguing", "where tourists sit at cafes",
                   "where there are signs for #roads#", "where there is #shop#"],
    "clothing_color": ["green", "sky-blue", "red", "orange", "white", "purple", "blue", "dark", "bright"],
    "clothing": ["scarf", "hat", "tie", "sash", "carnation", "jacket", "pants", "boots"],
    "shop": shops_text,
    "canals": canals,
    "roads": walks,
    "squares": squares,
    "photo": ["#photo_tag.get_photo#"],
    "photo_tag": ["PHOTO: "],
    "setting": ["#_.get_prefix#"],
    "atmosphere": ["#_.get_atmo#"],
    "map": ["#_.get_map#"]
}

In [66]:
# Main function.  Some globals here that need to be seen and modified by other functions and I didn't have time for
# refactoring well.

def generate_sentences(rules, otherlibs=[base_english, functions], howmany=10):
    global sentences  # needs to be seen by some of the code looking for context input to GPT2 model
    global counter  # used to choose clauses in some modifier functions; set by how far into 'howmany' we are.
    global length
    
    length = howmany
    sentences = []
    wordcount = 0
    
    grammar = tracery.Grammar(rules)
    for lib in otherlibs:
        grammar.add_modifiers(lib)
     
    for i in range(0, howmany):
        counter = i
        sent = grammar.flatten("#origin#")
        if len(sent.strip()) == 0:
            print("no text?")
        wordcount += len(sent.split(' '))
        print(sent)
        sentences.append(sent)
    print("Total words", wordcount)
    return sentences

In [68]:
# Run the loop over and over and over...  and write the sentences to the file.

maps = 0  # only run this if you start fresh - the number is needed for the filenames in the output.

sentences = generate_sentences(rules, howmany=700)

Look for a water taxi when you reach the Rio della Ca'di Dio.

Don't forget to veer right onto the Sottoportego del Servo over the bridge.

Setting the scene: Sottoportego del Servo is the place where the city’s rich merchants’ meetings are held. Frequently asked questions. When is the opening of the Canal? The opening of the Canal is at three times the day of the week of the year; it is also on the third Sunday of October. When is the closing of the Canal? The closing of the Canal is at three times the day of the week of the year; it is also on the third Sunday of October.

Some thoughts here: How many seats are in the galleries of the Canal Grande? There are five hundred seats in the public galleries. Do the gondolas cross the Canal to the Rio della Gondola? Yes, they do; the gondolas are a means of inter-water communication. Do the gondolas land on the Rialto Bridge? Yes, they do. The Rio della Gondola (Bridge of the Rialto) connects the Canal Grande to the Piazzetta. The Rialto Bri

Reminder here: They were the _mi città_ of the _città_ of the _città_ of the _città_ of the _città_ of the _città_. The new canals were, as you have guessed, the direct successors of the old ones. They were the direct descendants of the rio-fiorentino (which had been built and enlarged in the time of the old patrician class), and they were the direct descendants of the rio-foscari (which had been built and enlarged in the time of the new Venetian class). The rio-foscari were the original rulers of Venice, and they ruled Venice for a very long time.

Hire a boat at the Rio di Santa Fosca. While you relax in the boat, admire --But back to the point.

Now you will arrive at the Rio del Piombo. You see people talking, laughing, and reading. And as I sit here, I see the same old sense of fun in all these people's faces, and I know that everybody wants to be in Venice, but I don't know how. If I were to tell you that I knew, you'd laugh too. For one thing, I know that you don't want to belie

You will find the Campo San Severo. You see crowds gazing, and a general joy in the air. Many people are standing up and talking, and laughing heartily. On the left is a church, the Redentore, where a very great service is being held, and where the great Confraternity of St. Mark is being celebrated by a great host. To the right is a public gondola club with games and good company, where people are wandering about, and the young men and women are playing the part of piazza men, and passing round flattery and obsequiousity. To the of the Gondola stands the little church of the Piazzetta, which has a pretty pink and white dome.


[<img src='http://farm2.staticflickr.com/1765/41722505480_0dac00b929_m.jpg' alt='Flickr, by Carlo Raso'>](https://www.flickr.com/photos/70125105@N06/41722505480/)


You will need to walk without turning around and go for no less than 10 minutes.


<img src='images/image_6.jpg'>


Now, veer right after the passageway the Ramo Corte Lezze through the tour group.




[<img src='http://farm66.staticflickr.com/65535/48977260473_31dca10fd5_m.jpg' alt='Flickr, by ChuckPalmer {cepalm}'>](https://www.flickr.com/photos/chuckpalmer/48977260473/)


So you know: You will be directed to the Gondola. The pilot will then take you to the little boat, and you will be on the water. _In the Accademia_] In the room of the piano, in the centre of the piano, is a fine piece by Giorgione, St. Rocco in Glory. In the room of the wood-carving are some fine pieces by Titian and his pupils, including a fine "Assumption" by the great man himself. There is also a fine small wood-carving by the Florentine artist, Paolo Veneziano.

You now find yourself in the square the Campo Santi Giovanni e Paolo. From the square, you take a right, and after crossing the Ponte della Paglia, you turn to the to the church of the Redentore. This is one of the most beautiful churches in the world. Venice is rich with ecclesiastical history. The Redentore is the old church of the Venetians, whic

Some thoughts here: The elder Zeno was a monk and the younger a novice. The elder Zeno, as you see him, is a good priest and a great theologian. He had been in Rome for the same length of time as his predecessor, but had already made an impact on the history of the city. He was the first to apply the new system of liturgy to the Mass, and to introduce the four Evangelists. He was the first to give the Mass to the laity openly, and the first to present it to the people in such a way as to appeal to the heart and make it more responsive to the call to adoration of God.

Now take a right onto the Calle del Tragheto da la Madoneta for at least 2 minutes, while meandering to admire the views of --But I've gotten off-topic.

Now pass the tenth alleyway.


[<img src='http://farm66.staticflickr.com/65535/49453034701_28c68cd0d1_m.jpg' alt='Flickr, by Leo Kerner'>](https://www.flickr.com/photos/lkerner/49453034701/)


Then, go left on the Rio dei Mendicanti where you can smell pastries.

Don't f

...

You will find yourself in the square the Campo Santi Giovanni e Paolo.

Now, take a gondola at the Rio de San Nicolo dei Mendicoli. While you relax in the boat, admire the church, you receive a warm welcome from the brothers, who come from the breadth of the poorer sections of the city. You are now in a new and pleasant quarter, and you feel at ease, having found yourself in a place where you can work at your ease, and where you have room to do nothing.


[<img src='http://farm66.staticflickr.com/65535/49085544138_e8b83954d9_m.jpg' alt='Flickr, by MacClure'>](https://www.flickr.com/photos/37266013@N00/49085544138/)


Just an aside: A little way down the canal you see the beautiful old palace of the Doge Andrea Gritti, famous for his The Doge, a tall, lanky man of about my size, with a tall, slender beard, taught to read, and an easy smile, is one of the most agreeable persons I ever met with.

You will enter the square the Campo San Gallo. There are several reasons for going there: the

Some thoughts here: It is one of the most beautiful boats in the world. The driver is an excellent fellow by the name of Donato da Ponte, and will take you to and fro instantly. From the Portici to the Rialto we come to the church of San Giorgio, and then the campo of the Rialto family, and then the beautiful little church of San Vidal. The Rialto family The family which invented the first bank in Venice and the first coin of Venice, the Rialto, were, according to tradition, the originators of the city.


[<img src='http://farm66.staticflickr.com/65535/49450512521_71a33ec9a1_m.jpg' alt='Flickr, by rrrrrrraph'>](https://www.flickr.com/photos/rrrrrrraph/49450512521/)


Now after the sixth turn, turn right on the Rio de le Becarie.

Now take the second right after the passageway the Campo San Luca.

Here is a nice big square, the square the Campo Madonna dell'Orto. The Campo Madonna dell'Orto is the old market-place of the city, and used to be the main public square of Venice. The Campo M

Hire a boat at the Rio del Ghetto Nuovo. While you relax in the boat, admire the people in the cafes, you see little birds flying about, and lizards and parrots, and other little birds in the gondolas. It is all very romantic, and the people are very friendly. There is also a great deal of singing, and dancing. From the top of the tower, you have a beautiful view of the Piazza and the sea.

--But I've gotten off-topic.

Some thoughts here: The real charm of the day is the opera. Every year the opera is given in the church, and every year the opera is given on the Riva, so I was quite surprised to find that this year the opera was given on the Piazza only. I am rather too young to have missed it, but I should like to have, so I sat in the right transept, and listened to the first act, which was a fine performance by the playwright Antonio da Murano. It was a very serene and serenely beautiful performance, and the actors did their best, as one would wish they could have done in their you

Some thoughts here: Via del Sant'Apollinare 1 • Daily 8am–8pm • €5 • The Accademia • palazzoncontarini.it The Palazzo Contarini, which stands on the site of the Palazzo Loredan, was designed in the early thirteenth century by the Flavian family, and its most characteristic feature is the Renaissance-style capitals of the windows. There are a few surviving examples of their work – the capitals of the two chapels on the right are very rich, and a very early example of their capitals is on the facade of the church itself. The Palazzo Contarini has been beautifully restored and is the finest of the three palaces mentioned above.

Now, go left after the passageway the Ponte Vendramin and go past the dogs, --ok these are ones you can pet.

Now, take a gondola at the Rio Malpaga. Get off the boat after 9 minutes.


<img src='images/image_45.jpg'>


You will see the Biblioteca "Dante Alighieri" dell'Istituto di Studi Militari Marittimi, a library. You see it sprang. It is a cat. When you turn,


[<img src='http://farm66.staticflickr.com/65535/49450512521_71a33ec9a1_m.jpg' alt='Flickr, by rrrrrrraph'>](https://www.flickr.com/photos/rrrrrrraph/49450512521/)


Then, take the first turn right onto the Fondamenta del Forner.

Take a boat at the the Rio de la Guerra. Look for the cheaper water taxis first.


[<img src='http://farm66.staticflickr.com/65535/49280678386_d8e02afc9d_m.jpg' alt='Flickr, by Angelo A.Filippin'>](https://www.flickr.com/photos/angelofilippin/49280678386/)


Total words 25080


In [69]:
len(sentences)

700

In [70]:
# global vars for state/links

maps, counter, length

(49, 699, 700)

## Save the Results

In [71]:
# Write to a Fresh File in an Output Dir With a Copy of the Images

from distutils.dir_util import copy_tree

outputdir = "raw7"

def write_to(sentences, outputdir="raw_res"):
    if not os.path.exists(outputdir):
        os.mkdir(outputdir)
        print("Directory " , outputdir ,  " Created ")
    else:    
        print("Directory " , outputdir ,  " already exists")
    with open(outputdir + f"/results_{outputdir}.md", "w") as handle:
        for sent in sentences:
            handle.write(sent + "\n")
    copy_tree("images", outputdir + "/images")

In [72]:
write_to(sentences, outputdir=outputdir)

Directory  raw7  Created 


In [74]:
ls -al raw7

total 156
drwxr-xr-x 3 jupyter jupyter   4096 Jan 31 09:37 ./
drwxr-xr-x 8 jupyter jupyter   4096 Jan 31 09:37 ../
drwxr-xr-x 2 jupyter jupyter   4096 Jan 31 09:37 images/
-rw-r--r-- 1 jupyter jupyter 146006 Jan 31 09:37 results_raw7.md


### Final Paragraphs

In [77]:
endrules = {
    "origin": ["""
        \nAre you there yet, you wonder?\n\nFinally, after a few miles of walking, you enter #squares.get_square_desc#.\n#decorate#\nYou have definitely seen the cat here, who purrs a greeting from the top of the old well. On your left is #shop#.  The prices are reasonable and it's a little harder to find, which is why I recommend it.\n\nI hope you enjoy your stay in Venice! (You can take these directions with you.)\n#decorate#
        """],
    "shop": shops_text,
    "decorate": ["#photo#", "#map#", "#newpara#"],
    "squares": squares,
    "photo": ["#photo_tag.get_photo#"],
    "photo_tag": ["PHOTO: "],
    "map": ["#_.get_map#"],
    "newpara": ['\n\n']
                }
                

In [78]:
# let's just pick the best one for the end, frankly.

sentences = generate_sentences(endrules, howmany=5)


        
Are you there yet, you wonder?

Finally, after a few miles of walking, you enter the square the Campo de la Carità. This is the last place on the Via del Carmine that is not either part of the Venetian State, or part of the Byzantine realm. It is the spot where the ducal palace was erected, and where the long walls of the campanile still stand. The campanile was raised in 1206 to keep out the deluge of the deluge, which was then ravaging the city.

<img src='images/image_51.jpg'>

You have definitely seen the cat here, who purrs a greeting from the top of the old well. On your left is the hotel, Sina Centurion Palace.  The prices are reasonable and it's a little harder to find, which is why I recommend it.

I hope you enjoy your stay in Venice! (You can take these directions with you.)

[<img src='http://farm66.staticflickr.com/65535/49439242476_65272c9be7_m.jpg' alt='Flickr, by janetliz'>](https://www.flickr.com/photos/janetliz/49439242476/)

        

        
Are you there

In [80]:
sentences[4]

"\n        \nAre you there yet, you wonder?\n\nFinally, after a few miles of walking, you enter the square the Campo San Polo. There, on the west side of the square, is the Campo di San Rocco, on the site of the ancient church of San Rocco. The building is a Gothic pagoda, and the ground floor is dedicated to the saintly relics of the friars of the Dominican order.\n\n<img src='images/image_54.jpg'>\n\nYou have definitely seen the cat here, who purrs a greeting from the top of the old well. On your left is the restaurant, Osteria Ai do Gobbi.  The prices are reasonable and it's a little harder to find, which is why I recommend it.\n\nI hope you enjoy your stay in Venice! (You can take these directions with you.)\n\n<img src='images/image_55.jpg'>\n\n        "

In [81]:
with open(outputdir + "/results_raw7.md", "a") as handle:
    handle.write(sentences[4] + "\n")

In [82]:
ls images

image_0.jpg   image_21.jpg  image_33.jpg  image_45.jpg  image_57.jpg
image_10.jpg  image_22.jpg  image_34.jpg  image_46.jpg  image_58.jpg
image_11.jpg  image_23.jpg  image_35.jpg  image_47.jpg  image_59.jpg
image_12.jpg  image_24.jpg  image_36.jpg  image_48.jpg  image_5.jpg
image_13.jpg  image_25.jpg  image_37.jpg  image_49.jpg  image_60.jpg
image_14.jpg  image_26.jpg  image_38.jpg  image_4.jpg   image_6.jpg
image_15.jpg  image_27.jpg  image_39.jpg  image_50.jpg  image_7.jpg
image_16.jpg  image_28.jpg  image_3.jpg   image_51.jpg  image_8.jpg
image_17.jpg  image_29.jpg  image_40.jpg  image_52.jpg  image_9.jpg
image_18.jpg  image_2.jpg   image_41.jpg  image_53.jpg
image_19.jpg  image_30.jpg  image_42.jpg  image_54.jpg
image_1.jpg   image_31.jpg  image_43.jpg  image_55.jpg
image_20.jpg  image_32.jpg  image_44.jpg  image_56.jpg


In [85]:
cp images/image_54.jpg raw7/images/image_54.jpg

In [86]:
cp images/image_55.jpg raw7/images/image_55.jpg

In [ ]:
# ship it.